In [1]:
import numpy as np
import os
import cv2
import pandas as pd
import copy
import math

In [2]:
def load_data(dir_name = 'images'):    
    imgs = []
    files = os.listdir(dir_name)
    for filename in filter(lambda x: x.endswith('png'), files):
        img = cv2.imread(os.path.join(dir_name, filename), 0)
        imgs.append(img.astype(float))
    return imgs

In [3]:
def expend_img(input_img, window_size=2, stride=2):
    width = img.shape[1]
    height = img.shape[0]
    width_strides = width // stride if width % stride == 0 else width // stride + 1
    height_strides = height // stride if height % stride == 0 else height // stride + 1
    width_idx = stride * (width_strides - 1)
    height_idx = stride * (height_strides - 1)
    lack_w = (window_size - 1) - (width - 1 - width_idx)
    lack_h = (window_size - 1) - (height - 1 - height_idx)
    
    result = img.T.tolist()
    for i in range(lack_w):
        result.append((img.T[-lack_w:])[i].tolist())
    result = np.array(result).T
    
    result = result.tolist()
    for j in range(lack_h):
        result.append((result[-lack_h:])[j])
        
    return np.array(result)

In [4]:
def compute_choquet(img_wind, q):
    wind = img_wind.copy()
    wind.resize(wind.shape[0] * wind.shape[1])
    wind = wind.tolist()
    wind.append(0)
    wind.sort()
    
    result = 0
    for i in range(1, len(wind)):
        m = ((len(wind) - i) / (len(wind) - 1)) ** q
        result += (wind[i] - wind[i - 1]) * m
    
    return result

In [5]:
def CMin_compute(img_wind, q):
    wind = img_wind.copy() / 255
    wind.resize(wind.shape[0] * wind.shape[1])
    wind = wind.tolist()
    wind.append(0)
    wind.sort()
    
    result = 0
    for i in range(1, len(wind)):
        m = ((len(wind) - i) / (len(wind) - 1)) ** q
        result = result + (min(wind[i], m) - min(wind[i - 1], m))
    
    return result

In [6]:
def aggr(img, window_size=2, stride=2, q=0.1):
    max_result_img = []
    mean_result_img = []
    choquet_result_img = []
    CMin_result_img = []
    height_index = 0
    width_index = 0
    while height_index < img.shape[0]:
        max_result_img_row = []
        mean_result_img_row = []
        choquet_result_img_row = []
        CMin_result_img_row = []
        width_index = 0
        while width_index < img.shape[1]:
            max_result_img_row.append(np.max(img[height_index : height_index + window_size, width_index : width_index + window_size]))
            mean_result_img_row.append(np.mean(img[height_index : height_index + window_size, width_index : width_index + window_size]))
            choquet_result_img_row.append(
                compute_choquet(img[height_index : height_index + window_size, width_index : width_index + window_size], q)
            )
            CMin_result_img_row.append(
                255*CMin_compute(img[height_index : height_index + window_size, width_index : width_index + window_size], q)
            )
            width_index += stride
        max_result_img.append(max_result_img_row.copy())
        mean_result_img.append(mean_result_img_row.copy())
        choquet_result_img.append(choquet_result_img_row.copy())
        CMin_result_img.append(CMin_result_img_row.copy())
        height_index += stride
    
    return (np.array(max_result_img), np.array(mean_result_img), np.array(choquet_result_img), np.array(CMin_result_img))

In [7]:
def my_resize(img, shape, stride=3):
    result = np.zeros((shape[0], shape[1]))
    for i in range(shape[0]):
        for j in range(shape[1]):
            result[i][j] = img[i // stride][j // stride]
    return result

## Функции для расчёта метрик

In [8]:
def AD_metric(in_img, agrred_img):
    return np.sum(in_img - agrred_img) / (in_img.shape[0]*in_img.shape[1])

def SC_metric(in_img, agrred_img):
    return np.sum(in_img**2) / np.sum(agrred_img**2)

def NK_metric(in_img, agrred_img):
    return np.sum(in_img*agrred_img) / np.sum(in_img**2)

def MD_metric(in_img, agrred_img):
    return np.max(np.abs(in_img - agrred_img))

def NAE_metric(in_img, agrred_img):
    return np.sum(np.abs(in_img - agrred_img)) / np.sum(np.abs(in_img))

def MSE_metric(in_img, agrred_img):
    return np.sum((in_img - agrred_img)**2) / (in_img.shape[0] * in_img.shape[1])

def PSNR(in_img, agrred_img):
    param = 255**2 / MSE_metric(in_img, agrred_img)
    return 10*math.log(param, 10)



In [9]:
def executor():
    methods_names = ['max', 'mean', 'choquet', 'CMin']
    metrics_names = ['AD', 'SC', 'NK','MD', 'NAE', 'MSE', 'PSNR']
    methods = [max_aggr_imgs, mean_aggr_imgs, choquet_aggr_imgs, CMin_aggr_imgs]
    metrics = [AD_metric, SC_metric, NK_metric, MD_metric, NAE_metric, MSE_metric, PSNR]
    results = {str(i+1):{methods_names[j]:{metrics_names[k]: 
                             metrics[k](imgs[i], methods[j][i]) 
                                       for k in range(len(metrics))} 
                     for j in range(len(methods))} 
           for i in range(len(imgs))}
    
    everage_results = copy.deepcopy(results['1'])
    for method in methods_names:
        for metric in metrics_names:
            everage = 0
            for i in range(len(imgs)):
                everage += results[str(i+1)][method][metric]
                everage_results[method][metric] = everage / len(imgs)
            
    fine_ans = pd.DataFrame(columns=['max', 'mean', 'choquet', 'CMin', 'Name'])
    
    for i in range(len(imgs)):
        ans = pd.DataFrame(results[str(i+1)])
        ans['Name'] = i+1
        ans['Metric'] = ans.index
        fine_ans = pd.concat([fine_ans, ans], ignore_index=True, sort=True)
        
    final_result = fine_ans.groupby(['Name', 'Metric']).max()
    final_everage_result = final_result.groupby('Metric').mean()
    
    
    print(final_everage_result)
    
    writer = pd.ExcelWriter('results\\{}X{}stride{}\\Results({}).xlsx'.format(wind_size, wind_size, stride, q), engine='xlsxwriter')

    final_everage_result.to_excel(writer, 'Лист1')
    final_result.to_excel(writer, 'Лист2')

    writer.save()

## Реализация на наших изображениях

In [10]:
imgs = load_data('images')

In [12]:
max_aggr_imgs = []
mean_aggr_imgs = []
choquet_aggr_imgs = []
CMin_aggr_imgs = []
wind_size = 4
stride = 3
q = 0.7
for img in imgs:
    expended_img = expend_img(img, window_size=wind_size, stride=stride)
    maximum, mean, choquet, CMin = aggr(expended_img, window_size=wind_size, stride=stride, q=q)    
    max_aggr_imgs.append(my_resize(maximum, (img.shape[0], img.shape[1]), stride=stride))
    mean_aggr_imgs.append(my_resize(mean, (img.shape[0], img.shape[1]), stride=stride))
    choquet_aggr_imgs.append(my_resize(choquet, (img.shape[0], img.shape[1]), stride=stride))
    CMin_aggr_imgs.append(my_resize(CMin, (img.shape[0], img.shape[1]), stride=stride))

In [13]:
## расчёт метрик и их сохранение в xlsx таблицах
executor()

               CMin      choquet          max         mean
Metric                                                    
AD        -7.552049    -7.362377   -41.657028     0.067523
MD       161.622862   160.513092   223.750000   165.468750
MSE     1614.484001  1471.224679  5595.219289  1335.472390
NAE        0.237765     0.221859     0.438137     0.207390
NK         0.939662     0.952099     1.179954     0.899384
PSNR      16.845944    17.191453    11.749698    17.602046
SC         1.009011     0.992858     0.633817     1.139662


In [14]:
## сохранение изображений после агрегации
for i in range(len(imgs)):
    cv2.imwrite('results\\{}X{}stride{}\\{}max.png'.format(wind_size, wind_size, stride, i+1), max_aggr_imgs[i])
    cv2.imwrite('results\\{}X{}stride{}\\{}mean.png'.format(wind_size, wind_size, stride, i+1), mean_aggr_imgs[i])
    cv2.imwrite('results\\{}X{}stride{}\\{}choquet.png'.format(wind_size, wind_size, stride, i+1), choquet_aggr_imgs[i])
    cv2.imwrite('results\\{}X{}stride{}\\{}CMin.png'.format(wind_size, wind_size, stride, i+1), CMin_aggr_imgs[i])